<a href="https://colab.research.google.com/github/p4r41131Y4h000v3r/RIPE/blob/main/do_not_change_the_commands_but_I_am_keeps_fetchin_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You're absolutely right, and I apologize for the oversight. The issue was that the script wasn't keeping track of the previously processed block height, causing it to fetch the same block repeatedly.

Here's the corrected Python script that will fetch the most recent block and update automatically, without changing the `bitcoin-core.cli` commands:

In [ ]:
import json
import subprocess
import time
from datetime import datetime, timezone

# Function to get the block data from the Bitcoin Core node
def get_block_data(block_hash):
    try:
        result = subprocess.run(
            ["bitcoin-core.cli", "getblock", block_hash],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            check=True
        )
        return result.stdout.decode("utf-8")
    except subprocess.CalledProcessError as e:
        print(f"Error fetching block data: {e.stderr.decode('utf-8')}")
        return None

# Function to get transaction data using getrawtransaction
def get_transaction_data(tx_hash):
    try:
        result = subprocess.run(
            ["bitcoin-core.cli", "getrawtransaction", tx_hash, "true"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            check=True
        )
        return result.stdout.decode("utf-8")
    except subprocess.CalledProcessError as e:
        print(f"Error fetching transaction data: {e.stderr.decode('utf-8')}")
        return None

# Function to parse block data and extract relevant information
def parse_block_data(block_data):
    try:
        block = json.loads(block_data)

        # Extract block details
        block_hash = block.get("hash", "N/A")
        block_height = block.get("height", "N/A")
        timestamp = block.get("time", "N/A")
        num_transactions = block.get("nTx", "N/A")
        block_size = block.get("size", "N/A")
        difficulty = block.get("difficulty", "N/A")
        merkle_root = block.get("merkleroot", "N/A")
        version = block.get("versionHex", "N/A")
        nonce = block.get("nonce", "N/A")

        # Convert timestamp to human-readable date (timezone-aware)
        if timestamp != "N/A":
            date_time = datetime.fromtimestamp(timestamp, timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
        else:
            date_time = "N/A"

        # Get transaction value from the first transaction's outputs
        tx_value = "No value available"
        if "tx" in block and len(block["tx"]) > 0:
            first_tx_hash = block["tx"][0]
            # Fetch the transaction details using getrawtransaction
            transaction_data = get_transaction_data(first_tx_hash)
            if transaction_data:
                tx = json.loads(transaction_data)
                if "vout" in tx and len(tx["vout"]) > 0:
                    for output in tx["vout"]:
                        if "value" in output:
                            tx_value = output["value"]
                            break  # Once a value is found, exit loop

        return {
            "block_hash": block_hash,
            "block_height": block_height,
            "date_time": date_time,
            "tx_value": tx_value,
            "num_transactions": num_transactions,
            "block_size": block_size,
            "difficulty": difficulty,
            "merkle_root": merkle_root,
            "version": version,
            "nonce": nonce
        }
    except Exception as e:
        print(f"Error parsing block data: {e}")
        return None

# Function to get the current block height
def get_current_block_height():
    try:
        result = subprocess.run(
            ["bitcoin-core.cli", "getblockcount"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            check=True
        )
        current_height = int(result.stdout.decode("utf-8").strip())
        return current_height
    except subprocess.CalledProcessError as e:
        print(f"Error fetching current block height: {e.stderr.decode('utf-8')}")
        return None

# Function to get the current block hash for a given height
def get_block_hash_by_height(height):
    try:
        result = subprocess.run(
            ["bitcoin-core.cli", "getblockhash", str(height)],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            check=True
        )
        return result.stdout.decode("utf-8").strip()
    except subprocess.CalledProcessError as e:
        print(f"Error fetching block hash: {e.stderr.decode('utf-8')}")
        return None

# Main function to monitor and fetch block data
def monitor_block():
    last_processed_height = -1  # Initialize to an invalid height

    while True:
        # Get the current block height
        current_height = get_current_block_height()
        if current_height is None:
            time.sleep(10)  # Wait before trying again
            continue

        # Check if the current height is greater than the last processed height
        if current_height > last_processed_height:
            print(f"Current Block Height: {current_height}")

            # Get the current block hash
            current_hash = get_block_hash_by_height(current_height)
            if not current_hash:
                time.sleep(10)  # Wait before trying again
                continue
            print(f"Current Block Hash: {current_hash}")

            # Get the block data for the current block
            block_data = get_block_data(current_hash)
            if not block_data:
                time.sleep(10)  # Wait before trying again
                continue

            # Parse the block data and extract relevant information
            block_info = parse_block_data(block_data)
            if block_info:
                print(f"Block Hash: {block_info['block_hash']}")
                print(f"Block Height: {block_info['block_height']}")
                print(f"Date and Time: {block_info['date_time']}")
                print(f"Transaction Value: {block_info['tx_value']}")
                print(f"Number of Transactions: {block_info['num_transactions']}")
                print(f"Block Size: {block_info['block_size']} bytes")
                print(f"Difficulty: {block_info['difficulty']}")
                print(f"Merkle Root: {block_info['merkle_root']}")
                print(f"Version: {block_info['version']}")
                print(f"Nonce: {block_info['nonce']}")

                # Save the block info to a file
                with open("bitcoin_block_log.txt", "a") as file:
                    file.write(f"{datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S')} - {json.dumps(block_info)}\n")

            last_processed_height = current_height  # Update the last processed height
        else:
            print(f"Block {current_height} already processed. Waiting for new block.")

        # Wait before checking the next block (set interval in seconds)
        time.sleep(10)

# Run the block monitor
if __name__ == "__main__":
    monitor_block()

**Key Change:**

* **`last_processed_height`:** A variable `last_processed_height` is introduced to keep track of the last block height that was successfully processed.
* **Height Check:** Before fetching a block, the script now checks if `current_height` is greater than `last_processed_height`. If it is, the script fetches the block and updates `last_processed_height`. Otherwise, it waits for a new block.

This ensures that the script only fetches and processes new blocks as they are added to the blockchain.